In [56]:
import io
import chess
import chess.pgn
import chess.engine
import chess.svg
import time
import urllib
import json

engine = chess.engine.SimpleEngine.popen_uci("stockfish_15_x64_avx2.exe")

user='MartyMcShorty'
month_index = 0
game_index = 0
archives_url = f'https://api.chess.com/pub/player/{user}/games/archives'
archives_json =  json.loads(urllib.request.urlopen(archives_url).read())
month_url = archives_json['archives'][::-1][month_index]
month_json = json.loads(urllib.request.urlopen(month_url).read())
chessdotcom_game = month_json['games'][::-1][game_index]
game = chess.pgn.read_game(io.StringIO(chessdotcom_game['pgn']))

def convert_san(san, is_white=True):
    pieces = ['♚','♛','♝','♞','♟','♜']
    if is_white :
        pieces = ['♔','♕','♗','♘','♙','♖']

    if piece[0].upper() == 'K':
        piece = pieces[0]

    if piece[0].upper() == 'K':
        piece = pieces[0]
    if piece[0].upper() == 'Q':
        piece = pieces[1]
    if piece[0].upper() == 'B':
        piece = pieces[2]
    if piece[0].upper() == 'N':
        piece = pieces[3]
    if piece[0].upper() == 'R':
        piece = pieces[5]

    return piece

In [58]:



class ChessNode():
    def __init__(self, move_no, node, analysis, prev_chessnode):
        self.move_no = move_no
        self.node:chess.pgn.ChildNode = node
        self.analysis = analysis
        self.prev_chessnode = prev_chessnode

    def get_san(self, no=-1, formatted=False):
        
        board = self.node.board()
        if no == -1:
            # print(f'san for node before pop {board.fen()}')
            board.pop()
            # print(f'after  pop {board.fen()}')
            # print(f'move: {self.node.move}')
            if formatted:
                return convert_san(board.san(self.node.move))
            else:
                return board.san(self.node.move)
        else:
            # board.pop()
            # print(f' san for child line get {board.fen()}')
            move = self.analysis[no]['pv'][0]
            # print(f'move: {move}')
            if formatted:
                return convert_san(board.san(move))
            else:
                return board.san(move)
    

    def get_suggestions(self, formatted=False):
        suggestions = []
        if self.prev_chessnode:
            for i, analysis in enumerate(self.prev_chessnode.analysis):
                suggestions.append({
                    'san':self.prev_chessnode.get_san(i, formatted),
                    'score':self.prev_chessnode.get_score_int(i),
                    'score_str':self.prev_chessnode.get_score_str(i),
                })

        return suggestions

    def get_score_int(self, no=-1):
        score = 0
        if no == -1 and self.prev_chessnode:
            score = self.node.get_score_int(0)
        else:
            score = self.analysis[no]['score']
            if score.is_mate():
                score = score.pov(chess.WHITE).score(mate_score = 100000)
            else:
                score = score.pov(chess.WHITE).score()
        return score

    def get_score_str(self, no=-1):
        score = None
        if no == -1 and self.prev_chessnode:
            score = self.prev_chessnode.get_score_str(0)
        else:
            score = self.analysis[no]['score']
            if score.is_mate():
                score = f'M{score.pov(chess.WHITE).mate()}'
            else:
                score = f'{score.pov(chess.WHITE).score()/100:.1f}'

        return score

    def get_score_graph(self, no=-1):
        score = self.get_score_int(no)
        mult = 1
        if score < 0:
            mult = -1
        score = abs(score)
        if score > 100:
            score = 100

        # if score > 90000:
        #     if score < 99995:
        #         score = score - 

        score = score * mult
        return score 

moves_data = []
prev_chessnode  = None
moves = game.mainline_moves()
mainline = game.mainline()
for i,node in enumerate(game.mainline()):
    move_no= (i+2)/2
    # info = engine.analyse(node.board(), chess.engine.Limit(depth=14), multipv=3, info=chess.engine.INFO_ALL)
    info = engine.analyse(node.board(), chess.engine.Limit(time=0.01), multipv=3, info=chess.engine.INFO_ALL)
    
    chessnode = ChessNode(move_no, node, info, prev_chessnode)
    moves_data.append(chessnode)


    # print(f"no: {chessnode.move_no} move :{chessnode.get_san()} score: {chessnode.get_score_str()}")
    # for suggestion in chessnode.get_suggestions():
    #         print(f"    suggested move: {suggestion['san']} score: {suggestion['score_str']}")


    prev_chessnode = chessnode





In [ ]:
for chess_node in moves_data:
    print(f"no: {chess_node.move_no} move :{chess_node.get_san()} score: {chess_node.get_score_str()}")
    for suggestion in chess_node.get_suggestions():
        print(f"    suggested move: {suggestion['san']} score: {suggestion['score_str']}")

    


In [15]:
import tempfile
import os
import pathlib
os.makedirs('static/boards', exist_ok=True)

path = tempfile.mkdtemp(dir='static/boards')
p = pathlib.Path(path)
print(p.name)

tmptvmergko
